architecture gathered from: 

*   https://www.kaggle.com/drn01z3/end-to-end-baseline-with-u-net-keras
*   https://github.com/yihui-he/u-net/blob/master/train.py



In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Reshape, Dropout, concatenate, Conv2DTranspose, GaussianNoise
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd
import cv2
import os
import h5py
import random
from sklearn.metrics import jaccard_similarity_score

In [ ]:
## Load images.
dir = "/iahome/d/dm/dmattioli/Documents/IE6380_Project/data/Wire_Detection/synthetic/"
xTrain = os.listdir(dir + 'xTrain')
yTrain = os.listdir(dir + 'yTrain')

# Create random indices referencing images in data set.
N = np.array(xTrain).shape[0]
ind = list(range(0, N-10))
random.shuffle(ind)
rand_ind = ind

images = np.array(xTrain)[rand_ind]
masks = np.array(yTrain)[rand_ind]

## Format image data.
x = []
y = []

for f in images:
#  if f.endswith('.jpg'):
    img = cv2.imread(dir + 'xTrain/' + f, -1)
    new = np.expand_dims(img, axis=2)/255
    if new is not None:
      x.append(new) # append image to the stack

for m in masks:
#  if f.endswith('.jpg'):
    img = cv2.imread(dir + 'yTrain/' + m, -1)
    new = np.expand_dims(img, axis=2)/255
    if new is not None:
      y.append(new) # append image to the stack

print(np.array(x).shape)

In [ ]:
## Begin building model.
#smooth coefficients
smooth_j = 1e-12
smooth_d = 1

# Define jaccard coefficient metric
def jaccard_coef(y_true, y_pred):
   
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth_j) / (sum_ - intersection + smooth_j)

    return K.mean(jac)


# Define dice coefficient metric
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth_d) / (K.sum(y_true_f*y_true_f) + K.sum(y_pred_f*y_pred_f) + smooth_d)
  
    
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


# Function to create a CNN model with U-NET architecture 
def unet(rows, columns, channels):
    
    inputs = Input((rows, columns, channels))
    noise = GaussianNoise(0.1)(inputs)
    
    conv1 = Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', name='conv11')(noise)
    conv1 = Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', name='conv12')(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    drop1 = Dropout(0.3)(pool1)

    conv2 = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', name='conv21')(drop1)
    conv2 = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', name='conv22')(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    drop2 = Dropout(0.3)(pool2)
    
    conv3 = Conv2D(128, kernel_size=(3,3), padding='same', activation='relu', name='conv31')(drop2)
    conv3 = Conv2D(128, kernel_size=(3,3), padding='same', activation='relu', name='conv32')(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    drop3 = Dropout(0.3)(pool3)
    
    conv4 = Conv2D(256, kernel_size=(3,3), padding='same', activation='relu', name='conv41')(drop3)
    conv4 = Conv2D(256, kernel_size=(3,3), padding='same', activation='relu', name='conv42')(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    drop4 = Dropout(0.3)(pool4)
    
    conv5 = Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name='conv51')(drop4)
    conv5 = Conv2D(512, kernel_size=(3,3), padding='same', activation='relu', name='conv52')(conv5)
    
    up6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5)
    up6 = concatenate([up6, conv4], axis=3)
    conv6 = Conv2D(256, kernel_size=(3,3), padding='same', activation='relu', name='conv61')(up6)
    conv6 = Conv2D(256, kernel_size=(3,3), padding='same', activation='relu', name='conv62')(conv6)

    up7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = concatenate([up7, conv3], axis=3)
    conv7 = Conv2D(128, kernel_size=(3,3), padding='same', activation='relu', name='conv71')(up7)
    conv7 = Conv2D(128, kernel_size=(3,3), padding='same', activation='relu', name='conv72')(conv7)
    
    up8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = concatenate([up8, conv2], axis=3)
    conv8 = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', name='conv81')(up8)
    conv8 = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', name='conv82')(conv8)
    
    up9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = concatenate([up9, conv1], axis=3)
    conv9 = Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', name='conv91')(up9)
    conv9 = Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', name='conv92')(conv9)

    output = Conv2D(channels, kernel_size=(1,1), activation='sigmoid', name='output')(conv9)

    model = Model(inputs, output)
    
    return model

In [ ]:
# features = ImageDataGenerator(rescale=1./255)
# mask = ImageDataGenerator(rescale=1./255)

# feature_generator = features.flow_from_directory(
#         dir + 'xTrain',
#         target_size=(256, 256),
#         color_mode='grayscale',
#         batch_size=32,
#         class_mode=None,
#         shuffle=True
#         )        

# mask_generator = mask.flow_from_directory(
#         dir + 'yTrain',
#         target_size=(256, 256),
#         color_mode='grayscale',
#         batch_size=32,
#         class_mode=None,
#         shuffle=True
#         )

# train_generator = zip(feature_generator, mask_generator)

# K.clear_session()

# model_inputs = np.array(x).shape

# model = unet(model_inputs[1], model_inputs[2], model_inputs[3])

# opt = keras.optimizers.Adam(lr=0.0005)

# # model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[jaccard_coef, dice_coef, 'accuracy'])
# model.compile(optimizer=opt, loss=dice_coef_loss, metrics=[jaccard_coef, dice_coef, 'accuracy'])

# ## Train model to data.
# checkpoint = keras.callbacks.ModelCheckpoint('/iahome/d/dm/dmattioli/Documents/IE6380_Project/unet.h5', 
#                                              monitor='loss', verbose=1, save_best_only=True, mode='min') 
            
# model.fit_generator(
#         train_generator,
#         steps_per_epoch=2000,
#         epochs=50,
#         callbacks=[checkpoint]
#         )     

In [ ]:
K.clear_session()

model_inputs = np.array(x).shape
model = unet(model_inputs[1], model_inputs[2], model_inputs[3])
opt = keras.optimizers.Adam(lr=0.0005)

model.compile(optimizer=opt, loss=dice_coef_loss, metrics=[jaccard_coef, dice_coef, 'accuracy'])

## Train model to data
checkpoint = keras.callbacks.ModelCheckpoint('/iahome/d/dm/dmattioli/Documents/IE6380_Project/unet.h5',
                                             monitor='loss', verbose=1, save_best_only=True, mode='min') 

history = model.fit(np.array(x), np.array(y), validation_split=0.2,
                    batch_size=32, shuffle=True, epochs=50, callbacks=[checkpoint])

In [ ]:
print(history.history.keys())
print(history.history['val_loss'])